In [2]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
import pandas as pd

movies = pd.read_csv("./movieCleaned.csv")

In [5]:
movies

,index,budget,genres,id,keywords,original_language,original_title,overview,popularity,production_companies,...,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director,taggedDescription
0,0,237000000,Action Adventure Fantasy Science Fiction,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"['Ingenious Film Partners', 'Twentieth Century...",...,"['English', 'Español']",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"['Stephen E. Rivkin Editor', 'Rick Carter Prod...",James Cameron,"19995 In the 22nd century, a paraplegic Marine..."
1,1,300000000,Adventure Fantasy Action,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"['Walt Disney Pictures', 'Jerry Bruckheimer Fi...",...,['English'],Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"['Dariusz Wolski Director of Photography', 'Go...",Gore Verbinski,"285 Captain Barbossa, long believed to be dead..."
2,2,245000000,Action Adventure Crime,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"['Columbia Pictures', 'Danjaq', 'B24']",...,"['Français', 'English', 'Español', 'Italiano',...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"['Thomas Newman Original Music Composer', 'Sam...",Sam Mendes,206647 A cryptic message from Bond’s past send...
3,3,250000000,Action Crime Drama Thriller,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"['Legendary Pictures', 'Warner Bros.', 'DC Ent...",...,['English'],Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"['Hans Zimmer Original Music Composer', 'Charl...",Christopher Nolan,49026 Following the death of District Attorney...
4,4,260000000,Action Adventure Science Fiction,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,['Walt Disney Pictures'],...,['English'],Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"['Andrew Stanton Screenplay', 'Andrew Stanton ...",Andrew Stanton,"49529 John Carter is a war-weary, former milit..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3703,4792,20000,Crime Horror Mystery Thriller,36095,japan prostitute hotel based on novel hallucin...,ja,キュア,A wave of gruesome murders is sweeping Tokyo. ...,0.212443,['Daiei Studios'],...,['日本語'],Released,Madness. Terror. Murder.,Cure,7.4,63,Koji Yakusho Masato Hagiwara Tsuyoshi Ujiki An...,"['Kiyoshi Kurosawa Director', 'Kiyoshi Kurosaw...",Kiyoshi Kurosawa,36095 A wave of gruesome murders is sweeping T...
3704,4793,0,Drama,182291,confession hazing gang member latino lgbt,en,On The Downlow,Isaac and Angel are two young Latinos involved...,0.029757,['Iconoclast Films'],...,[],Released,Two gangs. One secret. One crossroad.,On The Downlow,6.0,2,Tony Sancho Michael Cortez Donato Cruz Felipe ...,"['Tadeo Garcia Director', 'Tadeo Garcia Story'...",Tadeo Garcia,182291 Isaac and Angel are two young Latinos i...
3705,4795,0,Drama,124606,gang audition police fake homeless actress,en,Bang,A young woman in L.A. is having a bad day: she...,0.918116,"['Asylum Films', 'FM Entertainment', 'Eagle Ey...",...,['English'],Released,Sometimes you've got to break the rules,Bang,6.0,1,Darling Narita Peter Greene Michael Newland Er...,"['Ash Baron-Cohen Director', 'Ash Baron-Cohen ...",Ash Baron-Cohen,124606 A young woman in L.A. is having a bad d...
3706,4796,7000,Sc

In [6]:
movies['taggedDescription'].to_csv('taggedDescription.txt', sep="\n", index=False, header=False)

In [7]:
raw_docs = TextLoader('taggedDescription.txt', encoding='utf-8').load()
textSplitter = CharacterTextSplitter(chunk_size=0, chunk_overlap=0, separator="\n")
documents = textSplitter.split_documents(raw_docs)

Created a chunk of size 181, which is longer than the specified 0
Created a chunk of size 180, which is longer than the specified 0
Created a chunk of size 247, which is longer than the specified 0
Created a chunk of size 434, which is longer than the specified 0
Created a chunk of size 348, which is longer than the specified 0
Created a chunk of size 291, which is longer than the specified 0
Created a chunk of size 518, which is longer than the specified 0
Created a chunk of size 400, which is longer than the specified 0
Created a chunk of size 178, which is longer than the specified 0
Created a chunk of size 382, which is longer than the specified 0
Created a chunk of size 381, which is longer than the specified 0
Created a chunk of size 132, which is longer than the specified 0
Created a chunk of size 392, which is longer than the specified 0
Created a chunk of size 299, which is longer than the specified 0
Created a chunk of size 442, which is longer than the specified 0
Created a 

In [8]:
documents[0]


Document(metadata={'source': 'taggedDescription.txt'}, page_content='19995 In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.')

In [66]:
from langchain.embeddings import HuggingFaceEmbeddings

# Load Hugging Face embedding model
embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
# embeddings_model = HuggingFaceEmbeddings(model_name="Alibaba-NLP/gte-modernbert-base")
# model = AutoModel.from_pretrained("NovaSearch/stella_en_400M_v5", trust_remote_code=True)
# Store documents and embeddings in Chroma
dbMovies = Chroma.from_documents(documents, embeddings_model)
# User query

InvalidDimensionException: Embedding dimension 768 does not match collection dimensionality 384

In [67]:
from transformers import pipeline
pipe = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")


Device set to use cpu


In [42]:
def retrieveRecommendations(query, topN):

    # Perform similarity search (Chroma will handle embedding generation internally)
    recommendations = dbMovies.similarity_search(query, k=topN)
    # Display results
    ids = []
    for recommendation in recommendations:
        ids.append(int(recommendation.page_content.split()[0].strip()))
    filteredMovies = movies[movies['id'].isin(ids)].copy()
    filteredMovies['id'] = pd.Categorical(filteredMovies['id'], categories=ids, ordered=True)
    filteredMovies = filteredMovies.sort_values('id')
    return filteredMovies

In [46]:
retrieveRecommendations(query='a professional dream diver going into peoples dreams to find information', topN=4)

,index,budget,genres,id,keywords,original_language,original_title,overview,popularity,production_companies,...,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director,taggedDescription
2992,3566,11000000,Horror Thriller Comedy,11284,amnesia nightmare alternate dimension psycholo...,en,Freddy's Dead: The Final Nightmare,"Just when you thought it was safe to sleep, Fr...",16.856231,['New Line Cinema'],...,['English'],Released,They saved the best for last.,Freddy's Dead: The Final Nightmare,5.1,272,Robert Englund Lisa Zane Shon Greenblatt Lezli...,"['Michael De Luca Screenplay', 'Michael De Luc...",Rachel Talalay,11284 Just when you thought it was safe to sle...
1772,1942,0,Fantasy Family,94352,clown music mimes cirque du soleil carnies,en,Cirque du Soleil: Worlds Away,An original story featuring performances by Ci...,5.987486,['Reel FX Creative Studios'],...,['English'],Released,Experience a Journey Like Never Before,Cirque du Soleil: Worlds Away,6.1,58,Erica Linz Igor Zaripov Matt Gillanders Jason ...,"['James Cameron Producer', 'Andrew Adamson Scr...",Andrew Adamson,94352 An original story featuring performances...
2570,2949,0,Comedy Romance,9027,sex indian lead dancing master new love porn a...,en,The Guru,Bored with Bollywood movies but fascinated wit...,4.093288,"['Universal Pictures', 'Studio Canal']",...,"['English', 'हिन्दी', 'Magyar', 'Español']",Released,"When he talks, women listen.",The Guru,5.1,75,Jimi Mistry Heather Graham Marisa Tomei Michae...,"['Tim Bevan Producer', 'Eric Fellner Producer'...",Daisy von Scherler Mayer,9027 Bored with Bollywood movies but fascinate...
1526,1651,0,Drama Thriller,28902,based on novel suicide attempt dream kidnappin...,en,In Dreams,Claire Cooper dreams strange things from time ...,3.226243,"['DreamWorks SKG', 'Amblin Entertainment']",...,['English'],Released,You don't have to sleep to dream,In Dreams,5.4,44,Annette Bening Aidan Quinn Stephen Rea Robert ...,"['Darius Khondji Director of Photography', 'Ja...",Neil Jordan,28902 Claire Cooper dreams strange things from...


In [9]:
movies.genres.value_counts()

genres
Drama                                           240
Comedy                                          215
Drama Romance                                   116
Comedy Romance                                  114
Comedy Drama                                     99
                                               ... 
Comedy Drama Fantasy Romance Science Fiction      1
Comedy Crime Science Fiction Thriller             1
War Drama Romance                                 1
Drama Thriller Romance                            1
Thriller Crime Horror                             1
Name: count, Length: 1018, dtype: int64

In [13]:
genreMapping = {
    'Drama': 'Drama',
    'Comedy': 'Comedy',
    'Comedy Romance': 'Romance',
    'Comedy Drama': 'Drama',
    'Comedy Drama Romance': 'Romance',
    'Horror': 'Horror',
    'Horror Thriller': 'Horror',
    'Drama Thriller': 'Drama',
    'Crime Drama Thriller': 'Drama',
    'Action Crime Thriller': 'Action',
    'Action Comedy': 'Action',
    'Adventure Action Thriller': 'Action',
}

movies['simpleGenres'] = movies['genres'].map(genreMapping)
movies[~(movies.simpleGenres.isna())]

,index,budget,genres,id,keywords,original_language,original_title,overview,popularity,production_companies,...,status,tagline,title,vote_average,vote_count,cast,crew,director,taggedDescription,simpleGenres
92,99,38000000,Action Crime Thriller,9799,street gang car race undercover auto-tuning lo...,en,The Fast and the Furious,Domenic Toretto is a Los Angeles street racer ...,6.909942,"['Universal Pictures', 'Original Film', 'Ardus...",...,Released,Live life 1/4 mile at a time.,The Fast and the Furious,6.6,3428,Paul Walker Vin Diesel Michelle Rodriguez Jord...,"['Ronna Kress Casting', 'Sanja Milkovic Hays C...",Rob Cohen,9799 Domenic Toretto is a Los Angeles street r...,Action
131,139,150000000,Adventure Action Thriller,956,berlin cia vatican white house secret identity,en,Mission: Impossible III,Retired from active duty to train new IMF agen...,63.079003,"['Paramount Pictures', 'Cruise/Wagner Producti...",...,Released,The Mission Begins 05:05:06.,Mission: Impossible III,6.5,2028,Tom Cruise Philip Seymour Hoffman Ving Rhames ...,"['Tom Cruise Producer', 'Colleen Atwood Costum...",J.J. Abrams,956 Retired from active duty to train new IMF ...,Action
139,147,140000000,Adventure Action Thriller,36669,laser british secret service secret service ag...,en,Die Another Day,Bond takes on a North Korean leader who underg...,54.159392,['Eon Productions'],...,Released,He’s never been cooler.,Die Another Day,5.8,1092,Pierce Brosnan Halle Berry Rosamund Pike Rick ...,"['Ian Fleming Characters', 'Lee Tamahori Direc...",Lee Tamahori,36669 Bond takes on a North Korean leader who ...,Action
147,155,100000000,Comedy,7552,based on novel desperation robber hold-up robb...,en,Fun with Dick and Jane,After Dick Harper loses his job at Globodyne i...,25.159168,"['Imagine Entertainment', 'Columbia Pictures C...",...,Released,Giving big businesses a run for their money!,Fun with Dick and Jane,5.9,627,Jim Carrey T\u00e9a Leoni Alec Baldwin Richard...,"['Debra Zane Casting', 'Julie Weiss Costume De...",Dean Parisot,7552 After Dick Harper loses his job at Globod...,Comedy
160,170,135000000,Adventure Action Thriller,36643,british mission oil heiress bilbao spain,en,The World Is Not Enough,"Greed, revenge, world dominance and high-tech ...",39.604363,"['Eon Productions', 'Metro-Goldwyn-Mayer (MGM)']",...,Released,As the countdown begins for the new millennium...,The World Is Not Enough,6.0,862,Pierce Brosnan Sophie Marceau Robert Carlyle D...,"['Ian Fleming Characters', 'Peter Lamont Produ...",Michael Apted,"36643 Greed, revenge, world dominance and high...",Action
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3701,4789,0,Drama,39851,addiction recovering drug addict estranged son,en,Clean,"After losing her husband to a heroin overdose,...",1.464566,[],...,Released,"When you don't have a choice, you change.",Clean,6.7,17,Maggie Cheung Nick Nolte B\u00e9atrice Dalle J...,"['Olivier Assayas Director', 'Olivier Assayas ...",Olivier Assayas,39851 After losing her husband to a heroin ove...,Drama
3702,4791,13,Horror,157185,home invasion,en,Tin Can Man,Recently dumped by his girlfirend for another ...,0.332679,"['Park Films', 'Camera Stylo Films']",...,Released,Everything You've Heard is True,Tin Can Man,2.0,1,Michael Parle Emma Eliza Regan Patrick O'Donne...,"['Ivan Kavanagh Director', 'Ivan Kavanagh Prod...",Ivan Kavanagh,157185 Recently dumped by his girlfirend for a...,Horror
3704,4793,0,Drama,182291,confession hazing gang member latino lgbt,en,On The Downlow,Isaac and Angel are two young Latinos involved...,0.029757,['Iconoclast Films'],...,Released,Two gangs. One secret. One crossroad.,On The Downlow,6.0,2,Tony Sancho Michael Cortez Donato Cruz Felipe ...,"['Tadeo Garcia Director', 'Tadeo Garcia Story'...",Tadeo Garcia,182291 Isaac and Angel are two young Latinos i...,Drama
3705,4795,0,Drama,124606,gang audition police fake homeless actress,en,Bang,A young woman in L.A. is having a bad day: she...,0.918116,"['Asylum Films

In [14]:
from transformers import pipeline

movieGenres = ['Drama', 'Comedy']

pipe = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=0)


Device set to use cpu


In [15]:
seq = movies.loc[movies['simpleGenres'] == 'Drama', 'overview'].reset_index(drop=True)[0]
pipe(seq, movieGenres)

{'sequence': 'Set in northern Australia before World War II, an English aristocrat who inherits a sprawling ranch reluctantly pacts with a stock-man in order to protect her new property from a takeover plot. As the pair drive 2,000 head of cattle over unforgiving landscape, they experience the bombing of Darwin, Australia, by Japanese forces firsthand.',
 'labels': ['Drama', 'Comedy'],
 'scores': [0.7342619299888611, 0.2657380998134613]}

In [22]:
import numpy as np
def generatePredictions(seq, genres):
    predictions = pipe(seq, genres)
    maxIndex = np.argmax(predictions['scores'])
    maxLabel = predictions['labels'][maxIndex]
    return maxLabel

In [17]:
movies.simpleGenres.value_counts()

simpleGenres
Drama      425
Comedy     215
Romance    192
Horror     126
Action      73
Name: count, dtype: int64

In [23]:
from tqdm import tqdm

actualCat = []
predCat = []

for i in tqdm(range(0,200)):
    seq = movies.loc[movies['simpleGenres'] == 'Drama', 'overview'].reset_index(drop=True)[i]
    predCat += [generatePredictions(seq, ['Drama', 'Comedy'])]
    actualCat += ['Drama']

100%|██████████| 200/200 [02:15<00:00,  1.47it/s]


In [25]:
for i in tqdm(range(0,200)):
    seq = movies.loc[movies['simpleGenres'] == 'Comedy', 'overview'].reset_index(drop=True)[i]
    predCat += [generatePredictions(seq, ['Drama', 'Comedy'])]
    actualCat += ['Comedy']

100%|██████████| 200/200 [01:53<00:00,  1.75it/s]


In [27]:
predictionsDF = pd.DataFrame({'actualCat': actualCat, 'predCat': predCat})
predictionsDF

,actualCat,predCat
0,Drama,Drama
1,Drama,Drama
2,Drama,Drama
3,Drama,Drama
4,Drama,Drama
...,...,...
395,Comedy,Comedy
396,Comedy,Drama
397,Comedy,Drama
398,Comedy,Drama


In [30]:
predictionsDF['correctPreds'] = np.where(predictionsDF['predCat'] == predictionsDF['actualCat'], 1, 0)
predictionsDF['correctPreds'].sum()/ len(predictionsDF)

np.float64(0.5875)

In [ ]:
ids = []
predCat = []

missingCats = movies.loc[movies['simpleGenres'].isna(), ['id', 'overview']].reset_index(drop=True)
genreClasses = list(genreMapping.keys())
for i in tqdm(range(0,len(missingCats))):
    predCat += [generatePredictions(missingCats['overview'][i], genreClasses)]
    ids += [missingCats['id'][i]]

  6%|▌         | 155/2677 [11:58<2:12:00,  3.14s/it]

In [ ]:
missingPred = pd.DataFrame({'id': ids, 'simpleGenres': predCat})
movies  = movies.merge(missingPred, on='id')
movies = (np.where(movies['simpleGenres'].isna(), movies['predCat'], movies['simpleGenres']))
movies.drop(columns=['predCat'], inplace=True)

In [ ]:
# from transformers import AutoTokenizer, AutoModel
# import torch
# from langchain_community.document_loaders import TextLoader
# from langchain_text_splitters import CharacterTextSplitter
# from langchain_community.vectorstores import Chroma
#
# # Load Hugging Face model
# model_name = "sentence-transformers/all-MiniLM-L6-v2"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModel.from_pretrained(model_name)
#
# def generate_embeddings(text):
#     inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
#     with torch.no_grad():
#         outputs = model(**inputs)
#         embeddings = outputs.last_hidden_state.mean(dim=1)
#     return embeddings
# raw_docs = TextLoader('taggedDescription.txt', encoding='utf-8').load()
# textSplitter = CharacterTextSplitter(chunk_size=0, chunk_overlap=0, separator="\n")
# documents = textSplitter.split_documents(raw_docs)
#
# # Generate embeddings for the movie descriptions
# movie_embeddings = [generate_embeddings(doc.page_content) for doc in documents]
#
# # Store the embeddings in Chroma
# db = Chroma.from_documents(documents, embeddings=movie_embeddings)